In [6]:
import Pkg
Pkg.add("Distributions")

   Resolving package versions...
   Installed Rmath ─────────────────── v0.7.0
   Installed Calculus ──────────────── v0.5.1
   Installed Rmath_jll ─────────────── v0.3.0+0
   Installed HypergeometricFunctions ─ v0.3.11
   Installed PDMats ────────────────── v0.11.16
   Installed DualNumbers ───────────── v0.6.8
   Installed StatsFuns ─────────────── v1.0.1
   Installed FillArrays ────────────── v0.13.5
   Installed QuadGK ────────────────── v2.5.0
   Installed DensityInterface ──────── v0.4.0
   Installed Distributions ─────────── v0.25.76
    Updating `~/.julia/environments/v1.8/Project.toml`
  [31c24e10] + Distributions v0.25.76
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [49dc2e85] + Calculus v0.5.1
  [b429d917] + DensityInterface v0.4.0
  [31c24e10] + Distributions v0.25.76
  [fa6b7ba4] + DualNumbers v0.6.8
  [1a297f60] + FillArrays v0.13.5
  [34004b35] + HypergeometricFunctions v0.3.11
  [90014a1f] + PDMats v0.11.16
  [1fd47b50] + QuadGK v2.5.0
  [79098fc4] + Rmath 

In [7]:
import Random: seed!, rand
import StatsBase: Histogram, fit, normalize, cor
#import LinearAlgebra: normalize
import DataStructures: counter, Accumulator
import Distributions: Binomial, Normal, truncated

N = 11111589 #number of throw of a die
#N = 50 #number of throw of a die
xdim = 6
ydim = 8

uniform_x = false
uniform_y = false
#See https://stats.stackexchange.com/questions/546544/are-discrete-random-variables-with-same-domain-and-uniform-probability-always
dependent_y = false

#seed!(378556)
if uniform_x == true
  x = rand(1:xdim,N) #uniform distribution
else
  x = round.(Int,rand(truncated(Normal(3,3),1,xdim),N)) #non-uniform distribution
end #if uniform_x == true

#DEBUG:
#println("x = ", x)
px = normalize(fit(Histogram, x, nbins=xdim), mode=:probability)

if uniform_x == true

  thpx = 1.0/xdim
  println("Theoretical probability: Th_P(x) = ", thpx)
  println("Marginal probability P(x):")
  display(px.weights)
  abs_diff_x = px.weights .- thpx
  println("Absolute difference P(x) - Th_P(x):")
  stddev_x = sqrt.( px.weights.*( (1.0 .- px.weights)./N ) )
  println("P(x) abs_err_x, standard_dev_x")
  display(hcat(px.weights, abs_diff_x, stddev_x))
  println()
  println("Relative errors P(x) vs Th_P(x)")
  display(abs_diff_x ./ thpx)
  println()

end #if uniform_x == true

#seed!(9385)
if uniform_y == true

  #if (dependent_y == true) && (uniform_x == true)
  if dependent_y == true
    coin = rand(Binomial(1, 0.5), N)
    #DEBUG
    #println("Coin:")
    #display(coin)
    y = (coin .* rand(1:ydim, N)) .+ ((1 .- coin) .* x)
  else
    y = rand(1:ydim, N) #uniform distribution
  end #if dependent_y == true

else
  y = round.(Int,rand(truncated(Normal(4,4),1,ydim),N)) #non-uniform distribution
end #if uniform_y == true

#DEBUG:
#println("y:")
#display(y)

py = normalize(fit(Histogram, y, 0:ydim, closed=:right), mode=:probability)


if uniform_y == true

  thpy = 1.0/ydim
  println("Theoretical probability: Th_P(y) = ", thpy)
  println("Marginal probability P(y):")
  display(py.weights)
  abs_diff_y = py.weights .- thpy
  println("Absolute difference P(y) - Th_P(y):")
  stddev_y = sqrt.( py.weights.*( (1.0 .- py.weights)./N ) )
  println("P(y) abs_err_y, standard_dev_y")
  display(hcat(py.weights, abs_diff_y, stddev_y))
  println()
  println("Relative errors P(y) vs Th_P(y)")
  display(abs_diff_y ./ thpy)
  println()

end #if uniform_y == true


# Create an ordered pair variable
orderedpairs = zip(x,y)

#DEBUG
#println("orderedpairs = ", orderedpairs)
#println("typeof(orderedpairs) = ", typeof(orderedpairs))

pxyapp = counter(orderedpairs) #count all the frequencies of the ordered pairs
#DEBUG
#println("pxyapp = ", pxyapp)
#println("pxyapp[(1,2)] = ", pxyapp[(1,2)])

#Joint-probability from ordered pairs relative frequencies
pxy = zeros(xdim,ydim)
for idx in pxyapp
  #CartesianIndex -> transform the ordered pair in a matrix indeces
  pxy[CartesianIndex(idx.first)] = idx.second/N
end

#DEBUG
println("Joint Probability P(X and Y)")
display(pxy)
println()

#Conditional probability from joint probability
px_given_y = pxy ./ py.weights' #' is the adjoint: since the array is Real, it is simply the transpose
py_given_x = pxy ./ px.weights

println("Conditional Probability - P(x|y):")
display(px_given_y)
println()
println("Conditional Probability - P(y|x):")
display(py_given_x)
println()

#DEBUG
println("Correlation(x,y) = ",cor(x,y))


Joint Probability P(X and Y)


6×8 Matrix{Float64}:
 0.00609373  0.013588   0.0149467  …  0.0136386  0.0116519   0.00499506
 0.0136612   0.0304093  0.0334182     0.0304277  0.0260257   0.0110837
 0.0143433   0.0320826  0.0351705     0.0321327  0.0275363   0.0117215
 0.0136243   0.0303857  0.0332663     0.0304913  0.0259679   0.0111658
 0.011536    0.0257617  0.0282854     0.025805   0.0219678   0.00942799
 0.00476521  0.0106096  0.011631   …  0.0105443  0.00908187  0.00387307


Conditional Probability - P(x|y):


6×8 Matrix{Float64}:
 0.0951791  0.0951293  0.0953733  …  0.0953482  0.0953264  0.0955676
 0.213377   0.212895   0.213237      0.212722   0.212921   0.212059
 0.224031   0.22461    0.224419      0.224642   0.22528    0.224262
 0.212801   0.21273    0.212269      0.213167   0.212449   0.213629
 0.180183   0.180357   0.180486      0.180404   0.179723   0.180381
 0.0744287  0.074278   0.0742161  …  0.0737161  0.0743006  0.0741014


Conditional Probability - P(y|x):


6×8 Matrix{Float64}:
 0.0639726  0.142648  0.156913  0.161768  …  0.143179  0.122323  0.0524386
 0.0641961  0.142898  0.157037  0.16144      0.142984  0.122299  0.0520841
 0.0638231  0.142757  0.156497  0.162268     0.14298   0.122528  0.0521571
 0.0640351  0.142814  0.156354  0.161873     0.143311  0.122051  0.0524799
 0.0639938  0.142908  0.156909  0.161869     0.143149  0.121863  0.0523002
 0.0642433  0.143037  0.156806  0.161703  …  0.142156  0.122439  0.0522158


Correlation(x,y) = -0.00017419050616785767
